In [1]:
import tensorflow as tf 
import os
from random import shuffle
import numpy as np
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

2022-03-20 20:56:58.797014: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


/root/fish_class/ENEL645_FinalProject_FishClassification/Scripts


In [2]:
os.chdir('/root/fish_class')
working_directory = os.getcwd()
print("working directory:", working_directory)

working directory: /root/fish_class
/root/fish_class


1. Loading Data and Preprocessing

In [58]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, # min-max Normalization, shifting pixel value to [0,1], max 255 to max of 1 (domain shift)
    validation_split=0.10
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(                                                    
    rescale=1./255 # Apply same normalization, not performing other preprocessing steps
)


In [59]:
train_images = train_generator.flow_from_directory(
    directory= './Data/Train_Val',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_directory(
    directory= './Data/Train_Val',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation' # Will only take 20% of the total data as the validation data
)

test_images = test_generator.flow_from_directory(
    directory= './Data/Test',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False, # We need to be able to generate metrics at the end
    seed=42
)

Found 8117 images belonging to 9 classes.
Found 900 images belonging to 9 classes.
Found 492 images belonging to 9 classes.


In [60]:
print("Training image shape:", train_images.image_shape)
print("Validation image shape:", val_images.image_shape)
print("Test image shape:", test_images.image_shape)

Training image shape: (224, 224, 3)
Validation image shape: (224, 224, 3)
Test image shape: (224, 224, 3)


In [61]:
train_images.class_indices

{'Black Sea Sprat': 0,
 'Gilt-Head Bream': 1,
 'Hourse Mackerel': 2,
 'Red Mullet': 3,
 'Red Sea Bream': 4,
 'Sea Bass': 5,
 'Shrimp': 6,
 'Striped Red Mullet': 7,
 'Trout': 8}

In [62]:
val_images.class_indices

{'Black Sea Sprat': 0,
 'Gilt-Head Bream': 1,
 'Hourse Mackerel': 2,
 'Red Mullet': 3,
 'Red Sea Bream': 4,
 'Sea Bass': 5,
 'Shrimp': 6,
 'Striped Red Mullet': 7,
 'Trout': 8}

In [63]:
test_images.class_indices

{'Black Sea Sprat': 0,
 'Gilt Head Bream': 1,
 'Horse Mackerel': 2,
 'Red Mullet': 3,
 'Red Sea Bream': 4,
 'Sea Bass': 5,
 'Shrimp': 6,
 'Striped Red Mullet': 7,
 'Trout': 8}

In [64]:
import tensorflow.keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

2. Defining VGG16 (CNN) Architecture

In [92]:
# Novel model - add descriptive layer names?
input = Input(shape =(224,224,3))
l1 = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.01))(input)
l2 = MaxPool2D(2,2)(l1)

l3 = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.01))(l2)
l4 = MaxPool2D(2,2)(l3)

l5 = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.01))(l4)
l6 = MaxPool2D(2,2)(l5)

l7 = Flatten()(l6)
l8 = Dense(64, activation='relu')(l7)
l9 = Dropout(0.2)(l8)
output = Dense(9, activation='softmax')(l9)
model = Model (inputs=input, outputs=output)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_57 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_58 (MaxPooling (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 52, 52, 128)       73856     
_________________________________________________________________
max_pooling2d_59 (MaxPooling (None, 26, 26, 128)       0  

3. Defining Schedulers and Callbacks

In [93]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 10) # Fine tune
checkpoint_path = "ENEL645_FinalProject_FishClassification/training_2_rof/cp.ckpt"
monitor = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss',
                                             verbose=1,save_best_only=True,
                                             save_weights_only=True,
                                             mode='min') # Only saves the best model (so far) in terms of min validation loss

lr_schedule = ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.1, patience=3, min_lr=0.0000001, verbose=1)
callbacks = [early_stop, monitor, lr_schedule]

4. Training Model

In [ ]:
history = model.fit(
    train_images, 
    validation_data=val_images, 
    epochs=50, # Fine tune
    callbacks=callbacks
)

Epoch 1/50
254/254 [==============================] - 96s 376ms/step - loss: 2.2940 - accuracy: 0.3977 - val_loss: 1.1073 - val_accuracy: 0.7011

Epoch 00001: val_loss improved from inf to 1.10732, saving model to ENEL645_FinalProject_FishClassification/training_2_rof/cp.ckpt
Epoch 2/50
254/254 [==============================] - 94s 370ms/step - loss: 0.8542 - accuracy: 0.7936 - val_loss: 0.6718 - val_accuracy: 0.8500

Epoch 00002: val_loss improved from 1.10732 to 0.67181, saving model to ENEL645_FinalProject_FishClassification/training_2_rof/cp.ckpt
Epoch 3/50
254/254 [==============================] - 94s 370ms/step - loss: 0.5203 - accuracy: 0.8961 - val_loss: 1.0548 - val_accuracy: 0.6967

Epoch 00003: val_loss did not improve from 0.67181
Epoch 4/50
254/254 [==============================] - 94s 368ms/step - loss: 0.4529 - accuracy: 0.9144 - val_loss: 0.6167 - val_accuracy: 0.8611

Epoch 00004: val_loss improved from 0.67181 to 0.61669, saving model to ENEL645_FinalProject_FishCl

In [132]:
np.save('ENEL645_FinalProject_FishClassification/history.npy', history.history)

In [ ]:
model.save('ENEL645_FinalProject_FishClassification/Model_rof')

In [ ]:
print("\n************************ COMPLETED TRAINING ************************")

5. Loading Best Model and Testing

In [89]:
model.load_weights(checkpoint_path)

In [133]:
history=np.load('ENEL645_FinalProject_FishClassification/history.npy', allow_pickle='TRUE').item()
print("Best training results:\n", history)

Best training results:
 {'loss': [1.1101419925689697], 'accuracy': [0.6045737862586975], 'val_loss': [1.1468136310577393], 'val_accuracy': [0.5604883432388306], 'lr': [1e-04]}


In [90]:
results = model.evaluate(test_images, verbose=1)

print("Categorical Cross Entropy: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

16/16 [==============================] - 27s 2s/step - loss: 3.2856 - accuracy: 0.3984
Categorical Cross Entropy: 3.28556
Test Accuracy: 39.84%


In [ ]:
print("\n************************ COMPLETED TESTING ************************")